# E.coli Data XML Data Scraping Phase 

In [1]:
import xml.etree.cElementTree as et
import pandas as pd
import numpy as np
import requests

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics import tsaplots

from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.debugger import set_trace

In [2]:
#Define "get value of node" function to handle errors where there is null data
def getvalueofnode(node):
    """ return node text or None """
    return node.text if node is not None else None

In [3]:
# retrieve the XML data from the URL
beaches_history = requests.get('http://app.toronto.ca/tpha/ws/beaches/history.xml?v=1.0&from=2009-06-01&to=2018-09-15').text 

In [4]:
#parse the XML
parsed_xml = et.fromstring(beaches_history)

In [5]:
#Define dataframe columns and create dataframe to receive parsed data
metacols = ['beachID','beachName','beachLat','beachLong']
dfcols = ['beachID', 'sampleDate', 'publishDate', 'eColiCount','beachAdvisory']
df_beaches_meta = pd.DataFrame(columns=metacols)
df_beaches_history = pd.DataFrame(columns=dfcols)

In [6]:
#Iterate through all elements and return the beach metadata via attributes
for node in parsed_xml.iter():
    beachID = node.attrib.get('id')
    beachName = node.attrib.get('name')
    beachLat = node.attrib.get('lat')
    beachLong = node.attrib.get('long')

    #Only append the result to the dataframe for elements where beachID is not blank
    if beachID is not None:
 
        df_beaches_meta = df_beaches_meta.append(
            pd.Series([beachID, beachName, beachLat, beachLong], index=metacols),
            ignore_index=True)

In [7]:
#Iterate through all elements and return the data we want via attributes
for node in parsed_xml.iter():
    beachID = node.attrib.get('beachId')
    sampleDate = node.find('sampleDate')
    publishDate = node.find('publishDate')
    eColiCount = node.find('eColiCount')
    beachAdvisory = node.find('beachAdvisory')

    #Only append the result to the dataframe for elements where beachID is not blank
    if beachID is not None:
 
        df_beaches_history = df_beaches_history.append(
            pd.Series([beachID, getvalueofnode(sampleDate), getvalueofnode(publishDate),
                        getvalueofnode(eColiCount),getvalueofnode(beachAdvisory)], index=dfcols),
            ignore_index=True)

In [8]:
#Replace empty data with np.nan values
df_beaches_history.replace(to_replace=[None], value=np.nan, inplace=True)

In [9]:
#Merge historical data with current data
beach_data_merged = results=df_beaches_history.merge(df_beaches_meta,on='beachID')

In [10]:
#Clean the beach data
beach_data_clean=df_beaches_history.drop(['beachAdvisory','publishDate'],axis=1)
beach_complete=beach_data_clean.pivot(index='sampleDate',columns='beachID',values='eColiCount')
beach_complete.index = pd.to_datetime(beach_complete.index)

In [11]:
beach_complete.head()

beachID,1,10,11,2,3,4,5,6,7,8,9
sampleDate,,,,,,,,,,,
2009-06-01,27,12,39,28,11,18,24,63,51,10,10
2009-06-02,10,14,52,16,10,18,22,18,19,10,10
2009-06-03,22,11,14,10,10,11,14,10,10,10,10
2009-06-04,26,10,11,10,10,11,11,10,10,10,10
2009-06-05,35,10,31,11,10,10,10,12,10,10,10


In [12]:
#Renaming the beach columns from their beachIDs to their names + beachIDs so that
#the beach columns are more easily recognized.
beach_complete = beach_complete.rename(columns={"1": "MCurtis-1", "10": "Bluffers-10", "11": "Rouge-11", 
                                       "2": "Sunnyside-2", "3": "Hanlans-3",
                                       "4": "Gibraltar-4", "5": "Centre-5",
                                       "6": "Wards-6", "7": "Cherry-7", "8": "Woodbine-8",
                                       "9": "KewBalmy-9"})
beach_complete.head()

beachID,MCurtis-1,Bluffers-10,Rouge-11,Sunnyside-2,Hanlans-3,Gibraltar-4,Centre-5,Wards-6,Cherry-7,Woodbine-8,KewBalmy-9
sampleDate,,,,,,,,,,,
2009-06-01,27,12,39,28,11,18,24,63,51,10,10
2009-06-02,10,14,52,16,10,18,22,18,19,10,10
2009-06-03,22,11,14,10,10,11,14,10,10,10,10
2009-06-04,26,10,11,10,10,11,11,10,10,10,10
2009-06-05,35,10,31,11,10,10,10,12,10,10,10


In [13]:
beach_complete.fillna(np.nan, inplace=True)

In [14]:
beach_complete.apply(pd.to_numeric)

beachID,MCurtis-1,Bluffers-10,Rouge-11,Sunnyside-2,Hanlans-3,Gibraltar-4,Centre-5,Wards-6,Cherry-7,Woodbine-8,KewBalmy-9
sampleDate,,,,,,,,,,,
2009-06-01,27.0,12.0,39.0,28.0,11.0,18.0,24.0,63.0,51.0,10.0,10.0
2009-06-02,10.0,14.0,52.0,16.0,10.0,18.0,22.0,18.0,19.0,10.0,10.0
2009-06-03,22.0,11.0,14.0,10.0,10.0,11.0,14.0,10.0,10.0,10.0,10.0
2009-06-04,26.0,10.0,11.0,10.0,10.0,11.0,11.0,10.0,10.0,10.0,10.0
2009-06-05,35.0,10.0,31.0,11.0,10.0,10.0,10.0,12.0,10.0,10.0,10.0
2009-06-06,133.0,13.0,29.0,18.0,12.0,10.0,12.0,16.0,10.0,12.0,10.0
2009-06-07,604.0,16.0,20.0,97.0,37.0,28.0,41.0,47.0,10.0,12.0,11.0
2009-06-08,405.0,17.0,99.0,142.0,31.0,33.0,43.0,39.0,10.0,10.0,12.0
2009-06-09,NaN,37.0,368.0,63.0,11.0,20.0,16.0,11.0,12.0,18.0,34.0


In [15]:
beach_complete.to_csv('beach_complete.csv',sep=',',na_rep=np.nan)